## Polytope Climate-DT Time Series example notebook for story nudging

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots.

Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [30]:
%%capture cap
%run ../desp-authentication.py

Exception: File `'../desp-authentication.py'` not found.

This will generate a token that can then be used by earthkit and polytope.

In [31]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

If you do not have eccodes installed please install eccodes using conda as it is a dependency of earthkit, or install earthkit via conda

* conda install eccodes -c conda-forge
* conda install earthkit-data -c conda-forge

In [32]:
import earthkit.data
from earthkit.plots.interactive import Chart
from polytope.api import Client

In [33]:
client = Client(
    address="polytope.lumi.apps.dte.destination-earth.eu")

request = {
     "activity": "story-nudging",
     "class": "d1",
     "dataset": "climate-dt",
     "experiment": "cont",
     "generation": "1",
     "levtype": "sfc",
     "date": "20180205/to/20180220",
     "model": "ifs-fesom",
     "expver": "0001",
     "param": "165",
     "realization": "1",
     "resolution": "high",
     "stream": "clte",
     "type": "fc",
     "time": "0000",
     "feature": {
         "type": "timeseries",
         "points": [[38, -9.5]],
         "time_axis": "date",
     }
 }

file = client.retrieve("destination-earth", request, "data/climate-dt-earthkit-fe-story-nudging.grib") 

DEBUG:polytope.api.Client.Q38268:Gathered Polytope client configuration:
{'address': 'polytope.lumi.apps.dte.destination-earth.eu',
 'insecure': False,
 'key_path': PosixPath('/Users/maes/.polytopeapirc'),
 'log_file': None,
 'log_level': 'DEBUG',
 'password': None,
 'port': None,
 'quiet': False,
 'skip_tls': False,
 'user_email': 'alice.sartorimonteirodebarros@ecmwf.int',
 'user_key': 'fd107a37bfb422bae0720ac8d68d0219',
 'username': 'maes',
 'verbose': False}
DEBUG:polytope.api.Client.Q38268:Creating Polytope client...
2025-07-10 11:19:08 - INFO - Sending request...
{'request': 'activity: story-nudging\n'
            'class: d1\n'
            'dataset: climate-dt\n'
            'date: 20180205/to/20180220\n'
            'experiment: cont\n'
            "expver: '0001'\n"
            'feature:\n'
            '  points:\n'
            '  - - 38\n'
            '    - -9.5\n'
            '  time_axis: date\n'
            '  type: timeseries\n'
            "generation: '1'\n"
            

In [34]:
data = earthkit.data.from_source("file", "data/climate-dt-earthkit-fe-story-nudging.grib") 

In [35]:
data._json()

{'type': 'CoverageCollection',
 'domainType': 'PointSeries',
 'coverages': [{'mars:metadata': {'activity': 'story-nudging',
    'class': 'd1',
    'dataset': 'climate-dt',
    'experiment': 'cont',
    'expver': '0001',
    'generation': '1',
    'levtype': 'sfc',
    'model': 'ifs-fesom',
    'realization': '1',
    'resolution': 'high',
    'stream': 'clte',
    'type': 'fc',
    'number': 0,
    'Forecast date': '2018-02-20 00:00:00Z'},
   'type': 'Coverage',
   'domain': {'type': 'Domain',
    'axes': {'x': {'values': [38.016284446058]},
     'y': {'values': [350.5078125]},
     'z': {'values': [0]},
     't': {'values': ['2018-02-05 00:00:00Z',
       '2018-02-06 00:00:00Z',
       '2018-02-07 00:00:00Z',
       '2018-02-08 00:00:00Z',
       '2018-02-09 00:00:00Z',
       '2018-02-10 00:00:00Z',
       '2018-02-11 00:00:00Z',
       '2018-02-12 00:00:00Z',
       '2018-02-13 00:00:00Z',
       '2018-02-14 00:00:00Z',
       '2018-02-15 00:00:00Z',
       '2018-02-16 00:00:00Z',
 

In [36]:
def location_to_string(location):
    """
    Converts latitude and longitude to a string representation with degrees
    and N/S/E/W.
    """
    (lat, lon) = location
    lat_dir = "N" if lat >= 0 else "S"
    lon_dir = "E" if lon >= 0 else "W"
    return f"{abs(lat):.2f}°{lat_dir}, {abs(lon):.2f}°{lon_dir}"

In [37]:
ds = data.to_xarray()
ds

<xarray.Dataset> Size: 368B
Dimensions:   (x: 1, y: 1, z: 1, number: 1, datetime: 1, t: 16)
Coordinates:
  * x         (x) float64 8B 38.02
  * y         (y) float64 8B 350.5
  * z         (z) int64 8B 0
  * number    (number) int64 8B 0
  * datetime  (datetime) <U20 80B '2018-02-20 00:00:00Z'
  * t         (t) datetime64[ns] 128B 2018-02-05 2018-02-06 ... 2018-02-20
Data variables:
    10u       (x, y, z, number, datetime, t) float64 128B -1.035 ... -1.723
Attributes: (12/14)
    activity:       story-nudging
    class:          d1
    dataset:        climate-dt
    experiment:     cont
    expver:         0001
    generation:     1
    ...             ...
    realization:    1
    resolution:     high
    stream:         clte
    type:           fc
    number:         0
    Forecast date:  2018-02-20 00:00:00Z

In [ ]:
TIME_FREQUENCY = "1d"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
LOCATION = ((38, -9.5))

chart = Chart()
chart.title(f"ECMWF ensemble meteogram at {location_to_string(LOCATION)}")
#chart.box(ds, time_frequency=TIME_FREQUENCY, quantiles=QUANTILES)
chart.line(ds, line_color='grey', time_frequency=TIME_FREQUENCY)
chart.show("png")  # Replace with chart.show() in an interactive session!